In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

from ugs import *

In [2]:
infile = "australia/data/AUS.vtk"
mesh = meshio.read(infile)

max_distance = 75000

In [3]:
# We need to define a way to calculate cost
def travel_cost(mesh, current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [4]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_from_point, mesh = mesh, travel_cost_function = travel_cost, max_distance = max_distance)

In [ ]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print get_from_point_in_mesh(2087)

## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [6]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [7]:
print "Total starting points available: ", points_above_sealevel.shape[0]

Total starting points available:  138726


In [ ]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

# For parallel execution, we chunk the data up into increments (inc), so we can monitor the progress
start = 0
inc = 200
stop = inc

# Run on 11 CPUs
p = Pool(11)

while start + inc <  points_above_sealevel.shape[0]:
    start_time = time.time()
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print "From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start]
    
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write("costed_aus.vtk", mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    
# If there are any points left over, process them now (this is just a hack, I should fix up the while loop to deal with them)    
start_time = time.time()
costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:])
print "From ", start, " to the end took ", time.time() - start_time, "seconds"
all_costs.extend(costs)
for i in all_costs:
    mesh.point_data['cost'][i[0]] = i[1]
meshio.write("costed_aus.vtk", mesh)

From  0  to  200  took  98.9378960133 seconds, starting with point 183353
From  200  to  400  took  102.309225798 seconds, starting with point 183153
From  400  to  600  took  106.129104137 seconds, starting with point 182953
From  600  to  800  took  106.374316931 seconds, starting with point 182753
From  800  to  1000  took  96.2294859886 seconds, starting with point 182553
From  1000  to  1200  took  92.492456913 seconds, starting with point 182353
From  1200  to  1400  took  103.19611907 seconds, starting with point 182153
From  1400  to  1600  took  98.0130019188 seconds, starting with point 181953
From  1600  to  1800  took  100.892317057 seconds, starting with point 181753
From  1800  to  2000  took  100.401367903 seconds, starting with point 181553
From  2000  to  2200  took  102.748775959 seconds, starting with point 181353
From  2200  to  2400  took  103.807703018 seconds, starting with point 181153
From  2400  to  2600  took  111.594316006 seconds, starting with point 180953

## Output

Show some of the results:

In [ ]:
print all_costs[:100]